In [28]:
import os
import requests
from dotenv import load_dotenv


In [ ]:

load_dotenv(override=True)

In [ ]:
def send_simple_message():
  	return requests.post(
  		"https://api.mailgun.net/v3/sandbox979b287399f848ed9122f9a12c836b17.mailgun.org/messages",
  		auth=("api", os.getenv('MAIL_GUN_API_KEY', 'KEY')),
  		data={"from": "Mailgun Sandbox <postmaster@sandbox979b287399f848ed9122f9a12c836b17.mailgun.org>",
			"to": "Son M Ngo <sonmngo@gmail.com>",
  			"subject": "Hello Son M Ngo",
  			"text": "Congratulations Son M Ngo, you just sent an email with Mailgun! You are truly awesome!"})

In [ ]:
send_simple_message()

In [47]:
def test_send_email(subject, content):
    return requests.post(
  		"https://api.mailgun.net/v3/sandbox979b287399f848ed9122f9a12c836b17.mailgun.org/messages",
  		auth=("api", os.getenv('MAIL_GUN_API_KEY', 'MAIL_GUN_KEY')),
  		data={"from": "Mailgun Sandbox <postmaster@sandbox979b287399f848ed9122f9a12c836b17.mailgun.org>",
			"to": "Son M Ngo <sonmngo@gmail.com>",
  			"subject": subject,
  			"text": content})

In [ ]:
test_send_email("Hello Son", "Congratulation on getting this email")

In [36]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import asyncio

In [34]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [39]:
model = "gpt-4o-mini"

In [40]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model= model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model= model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model= model
)

In [ ]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email ")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

In [ ]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


In [43]:
sales_picker = Agent(name="sales_picker", instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.", model=model)

In [ ]:
with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message )
    )

    outputs = [result.final_output for result in results]
    emails = "Cold sales emails:\n\n".join(outputs)
    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email: \n{best.final_output}")

In [50]:
import re

def extract_subject_and_body(email_text: str):
    """
    Extracts the subject line (value after 'Subject:') and returns
    both subject and body (without the subject line).
    """
    # Search for the Subject line
    match = re.search(r'^Subject:\s*(.+)$', email_text, re.MULTILINE)
    subject = match.group(1).strip() if match else None
    
    # Remove the Subject line from the body
    body = re.sub(r'^Subject:.*\n?', '', email_text, count=1, flags=re.MULTILINE).lstrip()
    
    return subject, body

In [ ]:
subject, body = extract_subject_and_body(best.final_output)

print("Subject:", subject)
print("Body:\n", body)

# Use of Tools

In [ ]:
# inspect agent
sales_agent1

In [55]:
@function_tool
def send_email(email: str):
    """ Send out an email with the given body to all sales prospects """
    subject, body = extract_subject_and_body(email)

    return requests.post(
  		"https://api.mailgun.net/v3/sandbox979b287399f848ed9122f9a12c836b17.mailgun.org/messages",
  		auth=("api", os.getenv('MAIL_GUN_API_KEY', 'MAIL_GUN_KEY')),
  		data={"from": "Mailgun Sandbox <postmaster@sandbox979b287399f848ed9122f9a12c836b17.mailgun.org>",
			"to": "Son M Ngo <sonmngo@gmail.com>",
  			"subject": subject,
  			"text": body})

In [56]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'email': {'title': 'Email', 'type': 'string'}}, 'required': ['email'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002667D9A5080>, strict_json_schema=True, is_enabled=True)

In [58]:
#Also, convert an Agent into a tool
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002667ECA99E0>, strict_json_schema=True, is_enabled=True)

In [59]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002667ECAB6A0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002667ECAB240>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

In [60]:
#Now, Time for Sales Manager to pick the best email and send email
instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across

In [61]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model=model)
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [62]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with given subjecy and HTL body to all sales prospects """
    return requests.post(
  		"https://api.mailgun.net/v3/sandbox979b287399f848ed9122f9a12c836b17.mailgun.org/messages",
  		auth=("api", os.getenv('MAIL_GUN_API_KEY', 'MAIL_GUN_KEY')),
  		data={"from": "Mailgun Sandbox <postmaster@sandbox979b287399f848ed9122f9a12c836b17.mailgun.org>",
			"to": "Son M Ngo <sonmngo@gmail.com>",
  			"subject": subject,
  			"html": html_body})

In [63]:
tools = [subject_tool, html_tool, send_html_email]

In [64]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model=model,
    handoff_description="Convert an email to HTML and send it")

In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

In [66]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)